In [4]:
from datasets import load_dataset
from functools import reduce
import pickle

In [5]:
dataset_name = "openbmb/UltraFeedback"
split_name = "train"

In [6]:
print(f"Loading dataset '{dataset_name}' (split: '{split_name}')...")

try:
    ds = load_dataset(dataset_name, split=split_name)
except Exception as e:
    print(f"Error loading dataset: {e}")
    print("Please ensure you have an internet connection, the 'datasets' library is installed (pip install datasets),")
    print("and the dataset name/split are correct.")

Loading dataset 'openbmb/UltraFeedback' (split: 'train')...


KeyboardInterrupt: 

In [7]:
def process_row(row):
    models = row["models"]
    completions = row["completions"]
    source = row["source"]
    instruction = row["instruction"]

    results = []

    for model, completion in zip(models, completions):
        answer = completion["response"]
        score = completion["overall_score"]
        grader = completion["model"]

        results.append({
            "source": source,
            "instruction": instruction,
            "model": model,
            "answer": answer,
            "score": score,
            "grader": grader,
        })
    
    return results

In [8]:
def convert_to_rows(df):
    raw = [process_row(row) for _, row in df.iterrows()]
    flat_map = lambda f, xs: reduce(lambda a, b: a + b, map(f, xs))
    flattened = flat_map(lambda x: x, raw)
    return flattened

In [ ]:
df = ds.to_pandas()
dataset = convert_to_rows(df)

with open('question_and_answers.pkl', 'wb') as f:
    pickle.dump(dataset, f)